In [1042]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools

In [1043]:
tfidf_df = pd.read_csv("tfidf_df_ngrams.csv")

In [1044]:
tfidf_df.sort_values(by='similarity', ascending=False, inplace=True)

In [1045]:
tfidf_df.reset_index(inplace=True, drop=True)

In [1046]:
# tfidf_df.tail()

In [1047]:
tfidf_df.shape

(101781, 3)

In [1048]:
tfidf_m = pd.DataFrame(np.sort(tfidf_df.iloc[:, :2])).duplicated()

In [1049]:
tfidf_df = tfidf_df[~tfidf_m]

In [1050]:
tfidf_df.reset_index(inplace=True, drop=True)

In [1051]:
# tfidf_df.head()
tfidf_df.shape

(64840, 3)

In [1052]:
tfidf_df = tfidf_df[tfidf_df.similarity > 0.274]

In [1053]:
tfidf_df.shape

(63221, 3)

In [1054]:
tfidf_df.similarity.describe()

count    63221.000000
mean         0.819840
std          0.187582
min          0.274244
25%          0.723082
50%          0.877739
75%          0.989859
max          1.000000
Name: similarity, dtype: float64

In [1055]:
submission_df = tfidf_df[['daltix_id_1', 'daltix_id_2']]

In [1056]:
## for testing doc2vec results.. Which is not a good solution..

# doc2vec_df = pd.read_csv("doc2vec_df.csv")
# doc2vec_df.sort_values(by='similarity', ascending=False, inplace=True)
# doc2vec_df.reset_index(inplace=True, drop=True)
# doc2vec_m = pd.DataFrame(np.sort(doc2vec_df.iloc[:, :2])).duplicated()
# doc2vec_df = doc2vec_df[~doc2vec_m]
# doc2vec_df.reset_index(inplace=True, drop=True)
# doc2vec_df.head()
# doc2vec_df.shape
# doc2vec_df = doc2vec_df[doc2vec_df.similarity > 0.5]
# doc2vec_df.shape
# submission_df = doc2vec_df[['daltix_id_1', 'daltix_id_2']]

In [1057]:
df_val = pd.read_csv("data/y_true.csv")
df_val.columns = ['daltix_id_1', 'daltix_id_2']

In [1058]:
len_validation = len(df_val)
len_submission = len(submission_df)
len_validation, len_submission

(59064, 63221)

In [1059]:
s1=pd.concat([submission_df,df_val])
df_val_ = df_val[['daltix_id_2', 'daltix_id_1']]
df_val_.columns = ['daltix_id_1', 'daltix_id_2']
s2=pd.concat([submission_df,df_val_])

In [1060]:
tp = len(s1[s1.duplicated()]) + len(s2[s2.duplicated()])
print(tp)
# 36733

36720


In [1061]:
fp = len(submission_df) - tp

In [1062]:
recall = tp/len_validation
precision = tp/len_submission
fpr = fp/len_submission

f1 = 2/((1/recall) + (1/precision))

In [1063]:
recall, precision, fpr, f1

(0.6216984965461194,
 0.5808196643520349,
 0.4191803356479651,
 0.6005642556323343)

In [1041]:
## To find out best threshold..  
    
tfidf_df = pd.read_csv("tfidf_df_ngrams.csv")
tfidf_df.sort_values(by='similarity', ascending=False, inplace=True)
tfidf_df.reset_index(inplace=True, drop=True)
tfidf_m = pd.DataFrame(np.sort(tfidf_df.iloc[:, :2])).duplicated()
tfidf_df = tfidf_df[~tfidf_m]
tfidf_df.reset_index(inplace=True, drop=True)

results = []
for i in np.arange(0.26, 0.28, 0.001):
    
    tfidf_df_ = tfidf_df[tfidf_df.similarity > i]
    submission_df = tfidf_df_[['daltix_id_1', 'daltix_id_2']]
    df_val = pd.read_csv("data/y_true.csv")
    df_val.columns = ['daltix_id_1', 'daltix_id_2']
    len_validation = len(df_val)
    len_submission = len(submission_df)

    s1=pd.concat([submission_df,df_val])
    df_val_ = df_val[['daltix_id_2', 'daltix_id_1']]
    df_val_.columns = ['daltix_id_1', 'daltix_id_2']
    s2=pd.concat([submission_df,df_val_])
    tp = len(s1[s1.duplicated()]) + len(s2[s2.duplicated()])
    fp = len(submission_df) - tp
    recall = tp/len_validation
    precision = tp/len_submission
    fpr = fp/len_submission

    f1 = 2/((1/recall) + (1/precision))
    results.append((i,f1))
results

[(0.26, 0.600252988941935),
 (0.261, 0.6002644552548239),
 (0.262, 0.600248171368861),
 (0.263, 0.6002351365914992),
 (0.264, 0.6002923236463701),
 (0.265, 0.6003250385881238),
 (0.266, 0.6003822842299587),
 (0.267, 0.6003741616084046),
 (0.268, 0.6003725672827099),
 (0.269, 0.6004167517875384),
 (0.27, 0.6004380731320594),
 (0.271, 0.6004626488691259),
 (0.272, 0.600498671571633),
 (0.273, 0.600506908674679),
 (0.274, 0.6005642556323343),
 (0.275, 0.6004923409092768),
 (0.276, 0.6004564155835658),
 (0.277, 0.6004744764397906),
 (0.278, 0.6004499897729596),
 (0.279, 0.6003927665493822),
 (0.28, 0.600373177079071)]